In [1]:
import os
from ase.io import read, write
from ase import Atoms

In [2]:
mol = read("../tests/example_data/david_test.xyz")
print(mol)
mol2 = Atoms(symbols=mol.get_chemical_symbols(), positions=mol.get_positions())
write("../tests/example_data/david_test.xyz", mol2)

Atoms(symbols='CNCN2C10NC6OH19', pbc=False)


In [3]:
# convert the molecule from xyz file to sdf
os.system("obabel ../tests/example_data/david_test.xyz -osdf -O ../tests/example_data/david_test.sdf")

1 molecule converted


0

In [4]:
# Import the necessary packages
from mace_openmm.mixed_system import MixedSystem
import torch
import logging
from typing import Optional, Union
from mace import tools

/rds/user/jhm72/hpc-work/openmmtools-dev/openmmtools/storage/iodrivers.py:88: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return '/'.join([path_part.strip('/ ') for path_part in split_path if path_part is not ''])
/rds/user/jhm72/hpc-work/openmmtools-dev/openmmtools/storage/iodrivers.py:703: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self._output_mode is 'a':


In [9]:
# the full system to be simulated
system_file = "../tests/example_data/david_test.sdf"
# the part of the system simulated with ML (in this case the whole system again)
ml_mol = "../tests/example_data/david_test.sdf"
model_path = "../tests/example_data/MACE_SPICE_larger.model"

resname="OpenBabel01062309583D"
potential = "mace"
temperature = 298.15
dtype = torch.float64
neighbour_list = "torch_nl_n2"

output_dir = "./small_molecule/md_out/"
system_type = "pure"

In [10]:
torch.set_default_dtype(dtype)

# I don't think this should be here
forcefields = [
    "amber/protein.ff14SB.xml",
    "amber/tip3p_standard.xml",
    "amber14/DNA.OL15.xml",
]

mixedSystem = MixedSystem(
    file=system_file,
    ml_mol=ml_mol,
    model_path=model_path,
    forcefields=forcefields, # not needed, default to anything in the case of pure? 
    resname=resname,
    padding=1.2, # not needed
    ionicStrength=0.0, # not needed
    nonbondedCutoff=1.0, # not needed
    potential=potential,
    temperature=temperature,
    dtype=dtype,
    neighbour_list=neighbour_list,
    output_dir=output_dir,
    system_type=system_type,
)

Initialized topology with (42, 3) positions
MACE model compiled


In [11]:
mixedSystem.run_mixed_md(steps=1000, interval=100, output_file="./output_md_test.pdb")

[W parser.cpp:3513] Warning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (function operator())
[W graph_fuser.cpp:104] Warning: operator() profile_node %512 : int[] = prim::profile_ivalue(%510)
 does not have profile information (function operator())


#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,0.10000000000000007,-115.55546337202293,33.365462752048096,0
200,0.20000000000000015,-105.82530814205796,67.14590271990329,4.54
300,0.3000000000000002,-93.20545692529697,86.74555800918337,4.57
400,0.4000000000000003,-79.8316648809265,88.97388545970507,4.56
500,0.5000000000000003,-74.4368489890595,109.10340446883207,4.56
600,0.6000000000000004,-71.35329481617238,141.4511877238335,4.56
700,0.7000000000000005,-67.47341847095346,133.6336511271457,4.56
800,0.8000000000000006,-55.178469580393326,136.58920174510786,4.57
900,0.9000000000000007,-50.69920722959874,148.56038294492703,4.57
1000,1.0000000000000007,-42.3886304779895,172.17199467380286,4.57
